<a href="https://www.kaggle.com/code/gpreda/kaggle-quadruple-and-triple-grandmasters-locations?scriptVersionId=150721567" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Kaggle Grandmasters with 3 and 4 tiers

Te Kaggle tiers are currently: Competitions, Datasets, Notebooks and Discussion. Some of the Kaggle Grandmasters have 4 (maximum) or 3 tiers.
For more details on the statistics of Kaggle tiers and rankings, see reference [1].  

Let's look to the geographical distribution of Kaggle triple and quadruple Grandmasters.  
Make sure to **click on the markers** on the map to see the details about each 3x or 4x Kaggle Grandmasters!!

We rerun periodically this Notebook, to capture the new entries in this exclusive club.

Note: **xhlulu** (triple Grandmaster) has no location information, and is not shown on this map.

In [1]:
import pandas as pd
import datetime
from IPython.display import Markdown
from IPython.display import Image
import folium
import numpy as np
from folium.plugins import MarkerCluster
from folium import Marker
n_users = pd.read_csv("../input/meta-kaggle/Users.csv")
master_achievements_df = pd.read_csv("../input/meta-kagglemaster-achievements-snapshot/MasterAchievements.csv")
master_achievements_df['GM_tier_count'] = master_achievements_df[master_achievements_df.astype(str) == 'grandmaster'].count(axis=1)
sel_gm_df = master_achievements_df[master_achievements_df['GM_tier_count'] > 2]
master_profiles_df = pd.read_csv("../input/meta-kagglemaster-achievements-snapshot/MasterProfiles.csv")
gm_profiles_df = sel_gm_df.merge(master_profiles_df)
gm_df = gm_profiles_df.loc[(~gm_profiles_df.Latitude.isna()) & (~gm_profiles_df.Longitude.isna())]

In [2]:
def popup_html(row):

    user_name = row['UserName']
    user_avatar = row['Avatar']
    user_address = row['Location']
    user_country = row['Country']
    user_co = row['Competitions']
    user_da = row['Datasets']
    user_no = row['Notebooks']
    user_di = row['Discussion']
    highest_rank = row['HighestRank']
    metadata = row['Metadata']
    title_type = row['GM_tier_count']

    gm_img_address = "https://www.kaggle.com/static/images/tiers/grandmaster@48.png"
    
    # set a width to 0 to hide 4th image if it is triple grandmaster only
    width="0"
    if title_type == 4:
        width="30"

    left_col_color = "#4166AA"
    right_col_color = "#f2f0d3"

    
    html = """<!DOCTYPE html>
    <html>
    <head>

    <center><img src=\"""" + user_avatar + """\" alt="logo" width=100 height=100 ></center>
    <h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(user_name) + """

    <img src=\"""" + gm_img_address + """\" alt="logo" width=30 height=30 >
    <img src=\"""" + gm_img_address + """\" alt="logo" width=30 height=30 >
    <img src=\"""" + gm_img_address + """\" alt="logo" width=30 height=30 >
    <img src=\"""" + gm_img_address + """\" alt="logo" width=""" + width + """ height=30 >

    <h6 style="margin-bottom:10"; width="200px">{}</h4>""".format("") + """  

    </head>
        <table style="height: 126px; width: 350px;">
    <tbody>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest Rank</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(highest_rank) + """
    </tr>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Competitions Tier</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_co) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Datasets Tier</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_da) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Notebooks Tier</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_no) + """
    </tr>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Discussions Tier</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_di) + """
    </tr>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Metadata</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(metadata) + """
    </tr>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_address) + """
    </tr>

    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Country</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(user_country) + """
    </tr>

    </tbody>
    </table>
    </html>
    """
    return html

In [3]:
gm_map = folium.Map(location=[0, 0], zoom_start=2)
width = 200
height = 500
gm3_popups, gm3_locations = [], []
gm4_popups, gm4_locations = [], []
for idx, row in gm_df.iterrows():

    if row['GM_tier_count'] == 3:
        gm3_locations.append([row['Latitude'], row['Longitude']])
        html = popup_html(row)
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        gm3_popups.append(popup)
    else:
        gm4_locations.append([row['Latitude'], row['Longitude']])
        html = popup_html(row)
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        gm4_popups.append(popup)        
    
gm3 = folium.FeatureGroup(name='Triple Grandmasters')
gm3.add_child(MarkerCluster(locations=gm3_locations, popups=gm3_popups))
gm_map.add_child(gm3)

gm4 = folium.FeatureGroup(name='Quadruple Grandmasters')
gm4.add_child(MarkerCluster(locations=gm4_locations, popups=gm4_popups))
gm_map.add_child(gm4)

## References  

[1] Carl McBride Ellis, Kaggle in Numbers, https://www.kaggle.com/code/carlmcbrideellis/kaggle-in-numbers  
[2] Use HTML in Folium Maps: A Comprehensive Guide for Data Scientists, My Data Talk, Towards Data Science, https://towardsdatascience.com/use-html-in-folium-maps-a-comprehensive-guide-for-data-scientists-3af10baf9190  